In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import random
import torch
import torchvision
import matplotlib.pyplot as plt
import time
import copy

from torchvision import transforms, models
from tqdm import tqdm_notebook as tqdm

data_root = './data/initial'
data_generated = './data/generated'
train_dir = './data/train'
val_dir = './data/val'
test_dir = './data/initial/test'

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [2]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(os.path.join(data_generated, 'train'), train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

In [3]:
def validate_model(model, loss, dataloader, device):
    was_training = model.training
    model.eval()
    running_loss = 0
    running_acc = 0
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            preds = model(inputs)
            loss_value = loss(preds, labels)
            preds_class = preds.argmax(dim=1)
            running_loss += loss_value.item()
            running_acc += (preds_class == labels.data).float().mean()
    epoch_loss = running_loss / len(dataloader)
    epoch_acc = running_acc / len(dataloader)
    if was_training:
        model.train()
    return epoch_loss, epoch_acc

In [4]:
def train_model(model, loss, optimizer, scheduler, num_epochs, eval_every_nth_batch):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        dataloader = train_dataloader
        model.train()

        running_loss = 0
        running_acc = 0
        batch_id = 0
        n_samples_passed = 0
        for inputs, labels in tqdm(dataloader):
            scheduler.step()
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            preds = model(inputs)
            loss_value = loss(preds, labels)
            preds_class = preds.argmax(dim=1)

            loss_value.backward()
            optimizer.step()

            running_loss += loss_value.item()
            running_acc += (preds_class == labels.data).float().mean()
            batch_id += 1
            if batch_id % eval_every_nth_batch == 0:
                train_loss = running_loss / eval_every_nth_batch
                train_acc = running_acc / eval_every_nth_batch
                val_loss, val_acc = validate_model(model, loss, val_dataloader, device)
                print('Train loss: {:.4f} Train acc: {:.4f}'.format(train_loss, train_acc), flush=True)
                print('>>> Val loss: {:.4f} Val acc: {:.4f}'.format(val_loss, val_acc), flush=True)
                running_loss = 0
                running_acc = 0

    return model

In [79]:
models.resnet18(pretrained=True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [5]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.__head = models.resnet18(pretrained=True)
        for param in self.__head.parameters():
            param.requires_grad = False
        #for param in self.__head.layer4[1].parameters():
        #    param.requires_grad = True
        #self.__head.fc = torch.nn.Linear(model.fc.in_features, 2)
        
        self.__bottom = torch.nn.Sequential()
        #self.__bottom.add_module("linear_1_dropout", torch.nn.Dropout(p=0.1))
        self.__bottom.add_module("linear_1", torch.nn.Linear(self.__head.fc.in_features, 2))
        
        self.__head.fc = self.__bottom
    
    def forward(self, x):
        return self.__head.forward(x)

model = Model()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, amsgrad=True, weight_decay=1e-6)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100500, gamma=0.1)

In [6]:
batch_size = 64
eval_every_nth_batch = 1
n_workers = 4
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=n_workers)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=n_workers)

train_model(model, loss, optimizer, scheduler, num_epochs=1, eval_every_nth_batch=eval_every_nth_batch);

Epoch 0/0:


A Jupyter Widget

Train loss: 0.8472 Train acc: 0.5625
>>> Val loss: 0.7485 Val acc: 0.4414
Train loss: 0.6670 Train acc: 0.5625
>>> Val loss: 0.6936 Val acc: 0.5534
Train loss: 0.6713 Train acc: 0.6250
>>> Val loss: 0.6803 Val acc: 0.5625
Train loss: 0.6787 Train acc: 0.6094
>>> Val loss: 0.6787 Val acc: 0.5599
Train loss: 0.6663 Train acc: 0.5781
>>> Val loss: 0.6673 Val acc: 0.5716
Train loss: 0.5403 Train acc: 0.7500
>>> Val loss: 0.6461 Val acc: 0.6276
Train loss: 0.5949 Train acc: 0.6094
>>> Val loss: 0.6301 Val acc: 0.6497
Train loss: 0.5761 Train acc: 0.7031
>>> Val loss: 0.6110 Val acc: 0.6758
Train loss: 0.5264 Train acc: 0.7812
>>> Val loss: 0.6010 Val acc: 0.6836
Train loss: 0.5360 Train acc: 0.7344
>>> Val loss: 0.5771 Val acc: 0.7057
Train loss: 0.5110 Train acc: 0.7656
>>> Val loss: 0.5585 Val acc: 0.7487
Train loss: 0.5004 Train acc: 0.7969
>>> Val loss: 0.5433 Val acc: 0.7643
Train loss: 0.4612 Train acc: 0.8125
>>> Val loss: 0.5332 Val acc: 0.7760
Train loss: 0.4817 Train acc: 0.8281
>

>>> Val loss: 0.4415 Val acc: 0.8438
Train loss: 0.2024 Train acc: 0.9375
>>> Val loss: 0.4427 Val acc: 0.8398
Train loss: 0.1234 Train acc: 1.0000
>>> Val loss: 0.4458 Val acc: 0.8359
Train loss: 0.1459 Train acc: 0.9844
>>> Val loss: 0.4503 Val acc: 0.8320
Train loss: 0.1064 Train acc: 1.0000
>>> Val loss: 0.4557 Val acc: 0.8229
Train loss: 0.1364 Train acc: 1.0000
>>> Val loss: 0.4645 Val acc: 0.7917
Train loss: 0.1700 Train acc: 0.9531
>>> Val loss: 0.4662 Val acc: 0.7878
Train loss: 0.1217 Train acc: 0.9688
>>> Val loss: 0.4568 Val acc: 0.8086
Train loss: 0.1650 Train acc: 0.9844
>>> Val loss: 0.4475 Val acc: 0.8320
Train loss: 0.1063 Train acc: 1.0000
>>> Val loss: 0.4428 Val acc: 0.8438
Train loss: 0.1131 Train acc: 0.9844
>>> Val loss: 0.4393 Val acc: 0.8477
Train loss: 0.1455 Train acc: 0.9688
>>> Val loss: 0.4373 Val acc: 0.8516
Train loss: 0.1078 Train acc: 1.0000
>>> Val loss: 0.4383 Val acc: 0.8424
Train loss: 0.1652 Train acc: 0.9375
>>> Val loss: 0.4363 Val acc: 0.8464
T

Train loss: 0.0954 Train acc: 0.9844
>>> Val loss: 0.4766 Val acc: 0.7734
Train loss: 0.1121 Train acc: 0.9688
>>> Val loss: 0.4710 Val acc: 0.7773
Train loss: 0.1046 Train acc: 0.9844
>>> Val loss: 0.4527 Val acc: 0.7969
Train loss: 0.0671 Train acc: 0.9844
>>> Val loss: 0.4362 Val acc: 0.8359
Train loss: 0.0785 Train acc: 1.0000
>>> Val loss: 0.4268 Val acc: 0.8555
Train loss: 0.0799 Train acc: 0.9844
>>> Val loss: 0.4243 Val acc: 0.8568
Train loss: 0.0845 Train acc: 0.9688
>>> Val loss: 0.4233 Val acc: 0.8607
Train loss: 0.0874 Train acc: 0.9688
>>> Val loss: 0.4228 Val acc: 0.8529
Train loss: 0.0503 Train acc: 1.0000
>>> Val loss: 0.4251 Val acc: 0.8633
Train loss: 0.0828 Train acc: 0.9844
>>> Val loss: 0.4237 Val acc: 0.8529
Train loss: 0.0751 Train acc: 0.9688
>>> Val loss: 0.4227 Val acc: 0.8594
Train loss: 0.0618 Train acc: 1.0000
>>> Val loss: 0.4303 Val acc: 0.8516
Train loss: 0.0760 Train acc: 0.9844
>>> Val loss: 0.4360 Val acc: 0.8281
Train loss: 0.1041 Train acc: 0.9531
>

>>> Val loss: 0.5076 Val acc: 0.7734
Train loss: 0.0384 Train acc: 1.0000
>>> Val loss: 0.4920 Val acc: 0.7773
Train loss: 0.0449 Train acc: 1.0000
>>> Val loss: 0.4808 Val acc: 0.7969
Train loss: 0.0526 Train acc: 0.9844
>>> Val loss: 0.4670 Val acc: 0.8008
Train loss: 0.0666 Train acc: 0.9844
>>> Val loss: 0.4616 Val acc: 0.8125
Train loss: 0.0501 Train acc: 0.9844
>>> Val loss: 0.4495 Val acc: 0.8320
Train loss: 0.0539 Train acc: 1.0000
>>> Val loss: 0.4447 Val acc: 0.8294
Train loss: 0.0396 Train acc: 1.0000
>>> Val loss: 0.4410 Val acc: 0.8529
Train loss: 0.0771 Train acc: 0.9688
>>> Val loss: 0.4387 Val acc: 0.8529
Train loss: 0.0606 Train acc: 0.9844
>>> Val loss: 0.4386 Val acc: 0.8464
Train loss: 0.0615 Train acc: 0.9688
>>> Val loss: 0.4382 Val acc: 0.8424
Train loss: 0.0622 Train acc: 0.9844
>>> Val loss: 0.4391 Val acc: 0.8411
Train loss: 0.0820 Train acc: 0.9688
>>> Val loss: 0.4401 Val acc: 0.8333
Train loss: 0.0583 Train acc: 0.9844
>>> Val loss: 0.4495 Val acc: 0.8177
T

KeyboardInterrupt: 

In [42]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = os.path.basename(self.imgs[index][0])
        path = path.split()[0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths(test_dir, val_transforms)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [43]:
model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

A Jupyter Widget

In [44]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [45]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

,label
id,
0000,dirty
0001,dirty
0002,dirty
0003,dirty
0004,dirty
0005,dirty


In [46]:
submission_df.to_csv('submission.csv')

# Data Preparation

In [12]:
class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

100%|███████████████████████████████████████| 20/20 [00:00<00:00, 172.42it/s]


Let's generate some samples

In [3]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

def prepare_dirs(root_path):
    shutil.rmtree(root_path, ignore_errors=True)
    os.makedirs(os.path.join(root_path, 'dirty'))
    os.makedirs(os.path.join(root_path, 'clean'))

def make_dataset(directory):
    ts = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ColorJitter(brightness=0.3, hue=0.25),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(180.0),
        transforms.RandomAffine(degrees=0, translate=(0.15, 0.15))
    ])
    return torchvision.datasets.ImageFolder(directory, ts)

def make_train_dataset():
    return make_dataset(train_dir)

def make_val_dataset():
    return make_dataset(val_dir)

def generate_images(n_amount, dataset_factory, dataset_type, root_path):
    path = os.path.join(root_path, dataset_type)
    prepare_dirs(path)
    name_pattern = '{{:0{}d}}.png'.format(len(str(n_amount)))
    class_path = {0: 'clean', 1: 'dirty'}
    image_id = 0
    while True:
        dataset = dataset_factory()
        data_iter = iter(dataset)
        for image, class_id in data_iter:
            if image_id >= n_amount:
                return
            image_name = name_pattern.format(image_id)
            image, class_id = next(data_iter)
            image.save(os.path.join(path, class_path[class_id], image_name), 'PNG')
            image_id += 1

In [4]:
n_gen_images_train = 262144
n_gen_images_val = 4096

generate_images(n_gen_images_train, make_train_dataset, 'train', data_generated)
#generate_images(n_gen_images_val, make_val_dataset, 'val', data_generated)